In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is B6F7-1CD2

 Directory of C:\Users\Anirudh\NNFLproj

11/24/2018  12:12 AM    <DIR>          .
11/24/2018  12:12 AM    <DIR>          ..
11/24/2018  12:12 AM    <DIR>          .ipynb_checkpoints
11/24/2018  12:12 AM            60,906 AllScratch(1).ipynb
11/19/2018  11:51 PM    <DIR>          clevr-iep-master
11/18/2018  07:20 PM    <DIR>          images
11/04/2018  05:55 PM        21,006,448 Quest_Answers.json
11/20/2018  12:36 AM                91 splitting.py
11/18/2018  07:32 PM    11,867,228,256 train_features.h5
11/19/2018  11:52 PM        25,928,032 train_questions.h5
11/19/2018  11:52 PM             1,613 vocab.json
               6 File(s) 11,914,225,346 bytes
               5 Dir(s)  17,517,096,960 bytes free


In [3]:
cd gdrive/

/content/gdrive


In [4]:
ls


'My Drive'/


In [5]:
cd My\ Drive

/content/gdrive/My Drive


In [7]:
cd NNFL\ Project

/content/gdrive/My Drive/NNFL Project


### CREATING IMAGE PRE-MODEL (4096 OUTPUT) ###

In [0]:
# from keras.applications.vgg16 import VGG16

# base_model = VGG16()
# # print(base_model.summary())

# # get output of penultimate layer
# from keras.models import Model
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)

In [0]:
# from keras.preprocessing.image import load_img
# from keras.preprocessing.image import img_to_array
# from keras.applications.vgg16 import preprocess_input

# # # load an image from file
# # image = load_img('Coffee-Mug.jpg', target_size=(224, 224))
# # # convert the image pixels to a numpy array
# # image = img_to_array(image)
# # # reshape data for the model
# # image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# # # prepare the image for the VGG model
# # image = preprocess_input(image)

# # # predict the probability across all output classes
# # yhat = model.predict(image)
# # print(yhat.shape)

### MAKE IMAGE DATA DICT ###
##### key: image name
##### value: vector of shape [1,4096], output of VGGNet

In [9]:
import os
import cv2
import pickle
import numpy as np


# imagedictv2: Images of size 100x100

if(os.path.isfile('image_dict.pickle')):
    with open('image_dict.pickle', 'rb') as handle:
        image_dict = pickle.load(handle)
        
else:
  folder = 'images/'

  image_dict_V2 = {}        

  skipped = []
  i=0

  for filename in os.listdir(folder):
      if(i%50 == 0):
          print("{} images processed".format(i))
      i+=1
      if filename in image_dict_V2:
          # print ("already in dict - moving on")
          continue
      try:
          # load an image from file
          image = cv2.imread(os.path.join(folder, filename))
      except:
          print("Error reading file: {}!!!".format(filename))
          skipped.append(filename)
          continue
      if image is not None:
          resized_image = cv2.resize(image, (100, 100)) 
          image_dict_V2[filename] = resized_image
      else:
          skipped.append(filename)

  print("{} files skipped:".format(len(skipped)))
  for f in skipped:
      print("    {}".format(f))
  print("dict created")

# # Save dict pickle

  print("Saving image_dict_V2.pickle")
  with open('image_dict_V2.pickle', 'wb') as handle:
      pickle.dump(image_dict_V2, handle, protocol=pickle.HIGHEST_PROTOCOL)
      print('image_dict_V2.pickle saved')



0 images processed
50 images processed
100 images processed
150 images processed
200 images processed
250 images processed
300 images processed
350 images processed
400 images processed
450 images processed
500 images processed
550 images processed
600 images processed
650 images processed
700 images processed
750 images processed
800 images processed
850 images processed
900 images processed
950 images processed
1000 images processed
1050 images processed
1100 images processed
1150 images processed
1200 images processed
1250 images processed
1300 images processed
1350 images processed
1400 images processed
1450 images processed
1500 images processed
1550 images processed
1600 images processed
1650 images processed
1700 images processed
1750 images processed
1800 images processed
1850 images processed
1900 images processed
1950 images processed
2000 images processed
2050 images processed
2100 images processed
2150 images processed
2200 images processed
2250 images processed
2300 images

In [11]:
# # Save dict pickle
# import pickle

# with open('image_dict.pickle', 'wb') as handle:
#     pickle.dump(images, handle, protocol=pickle.HIGHEST_PROTOCOL)
#     print('image_dict.pickle saved')

### COMBINING MODELS ###

In [5]:
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Dropout, LSTM, Flatten, Embedding, Multiply
# from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D

# def img_model(dropout_rate):
#     print("Creating image model...")
#     model = Sequential()
#     model.add(Dense(1024, input_dim=4096, activation='tanh'))
#     return model

# def Word2VecModel(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate):
#     print("Creating text model...")
#     model = Sequential()
#     model.add(Embedding(num_words, embedding_dim, 
#         weights=[embedding_matrix], input_length=seq_length, trainable=False))
#     model.add(LSTM(units=512, return_sequences=True, input_shape=(seq_length, embedding_dim)))
#     model.add(Dropout(dropout_rate))
#     model.add(LSTM(units=512, return_sequences=False))
#     model.add(Dropout(dropout_rate))
#     model.add(Dense(1024, activation='tanh'))
#     return model

# def vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
#     vgg_model = img_model(dropout_rate)
#     lstm_model = Word2VecModel(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate)
#     print("Merging final model...")
#     fc_model = Sequential()
#     # fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))
#     fc_model.add(Multiply([vgg_model, lstm_model]))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(1000, activation='tanh'))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(num_classes, activation='softmax'))
#     fc_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
#         metrics=['accuracy'])
#     return fc_model

### CREATE QUESTION DATA ###

In [13]:
# create 2 lists
# one stores image ids
# one stores question

imageNamesX  = []
questionsNLX = []
answers = []

import json
QAs = json.load(open("Quest_Answers.json", 'r'))['quest_answers']

for QA in QAs:
    img_name = QA["Image"]+".png"
    ques = QA["Question"]
    ans = QA["Answer"]
    
    if img_name not in image_dict_V2:
        print("Skipping {} - not found in dict".format(img))
        continue
    
    imageNamesX.append(img_name)
    questionsNLX.append(ques)
    answers.append(ans)

In [14]:
### NOW do word embeddings for questions
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 81
tokenizer = Tokenizer(num_words=num_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(questionsNLX)
questionsX = tokenizer.texts_to_sequences(questionsNLX)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_length_of_text = 200
questionsX = pad_sequences(questionsX, maxlen=max_length_of_text)

print("Saving word_index.pickle")
import pickle
with open('word_index.pickle', 'wb') as handle:
    pickle.dump(word_index, handle, protocol=pickle.HIGHEST_PROTOCOL)
print("Saved.")

Using TensorFlow backend.
C:\Users\Anirudh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Anirudh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Anirudh\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Anirudh\Anaconda3\lib\site-packages\tensorflow\python\framework\d

Found 80 unique tokens.
Saving word_index.pickle
Saved.


In [16]:
# vector embeddings

embeddings_index = {}
    
EMBEDDING_DIM = 200

embedding_matrix = None
if(os.path.isfile('embedding_matrix.pickle')):
    print(">> Embedding Matrix Pickle found...")
    with open('embedding_matrix.pickle', 'rb') as handle:
        embedding_matrix = pickle.load(handle)
    print(">>> loaded!")
else:
    f = open('glove.6B.200d.txt', encoding="utf8")

    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    print('Found %s word vectors.' % len(embeddings_index))

    embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
            
    with open('embedding_matrix.pickle', 'wb') as handle:
        pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print('embedding_matrix.pickle saved')

Found 400000 word vectors.
embedding_matrix.pickle saved


In [17]:
# One hot encode answers

## ONE HOT
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

data = answers
values = array(data)
# print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

Y = onehot_encoded

print("Saving label_encoder.pickle")
import pickle
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('label_encoder.pickle saved')

# print(onehot_encoded)
# invert first example
# inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
# print(inverted)

def decode_predictions(label_encoder, predictions):
    texts = []
    for p in predictions:
        text = label_encoder.inverse_transform(argmax(p))
        texts.append(text)
    return texts

label_encoder.classes_

Saving label_encoder.pickle
label_encoder.pickle saved


array(['0', '1', '2', '3', '4', '5', '6', '7', '8', 'False', 'True',
       'blue', 'brown', 'cube', 'cyan', 'cylinder', 'gray', 'green',
       'large', 'metal', 'purple', 'red', 'rubber', 'small', 'sphere',
       'yellow'], dtype='<U11')

In [18]:

# Split quesionsX
from sklearn.model_selection import train_test_split
imageNamesX_train, imageNamesX_test, questionsX_train, questionsX_test, Y_train, Y_test = train_test_split(imageNamesX, questionsX, Y, test_size=0.1, random_state=42)

### Data generator ###

In [19]:
def generator(image_dict, img_names, questions, labels, batch_size):
    
    q_ptr = 0
    while True:
        image_inp = []
        q_inp = []
        batch_labels = []
        for i in range(batch_size):
            if q_ptr == len(questions):
                q_ptr = 0
            index = q_ptr
#             import random
#             index= random.randint(0, len(questions)-1)
            # print(imageNamesX[q_ptr].shape)
            # print(questionsX[q_ptr])
            image_inp.append(image_dict[img_names[index]])
            q_inp.append(questions[index])
            batch_labels.append(labels[index])
            q_ptr+=1
            
        yield [np.array(image_inp), np.array(q_inp)], np.array(batch_labels)

In [20]:
def time_distributed_dense(x, w, b=None, dropout=None,
                           input_dim=None, output_dim=None, timesteps=None):
    '''Apply y.w + b for every temporal slice y of x.
    '''
    if not input_dim:
        # won't work with TensorFlow
        input_dim = K.shape(x)[2]
    if not timesteps:
        # won't work with TensorFlow
        timesteps = K.shape(x)[1]
    if not output_dim:
        # won't work with TensorFlow
        output_dim = K.shape(w)[1]

    if dropout:
        # apply the same dropout pattern at every timestep
        ones = K.ones_like(K.reshape(x[:, 0, :], (-1, input_dim)))
        dropout_matrix = K.dropout(ones, dropout)
        expanded_dropout_matrix = K.repeat(dropout_matrix, timesteps)
        x *= expanded_dropout_matrix

    # collapse time dimension and batch dimension together
    x = K.reshape(x, (-1, input_dim))

    x = K.dot(x, w)
    if b:
        x = x + b
    # reshape to 3D tensor
    x = K.reshape(x, (-1, timesteps, output_dim))
    return x

In [0]:
# def generator2(image_dict, img_names, questions, labels, batch_size):
    
#     q_ptr = 0
#     while True:
#         image_inp = []
#         q_inp = []
#         batch_labels = []
#         for i in range(batch_size):
#             if q_ptr == len(questions):
#                 q_ptr = 0
#             index = q_ptr
#             # print(imgsX2[q_ptr].shape)
#             # print(quesX2_sq[q_ptr])
#             image_inp.append(image_dict[img_names[index]])
#             q_inp.append(questions[index])
#             batch_labels.append(Y[index])
#             q_ptr+=1
#         yield [np.array(image_inp), np.array(q_inp)], np.array(batch_labels)

In [21]:
import tensorflow as tf
from keras import backend as K
from keras import regularizers, constraints, initializers, activations
from keras.layers.recurrent import Recurrent
from keras.engine import InputSpec

tfPrint = lambda d, T: tf.Print(input_=T, data=[T, tf.shape(T)], message=d)

class AttentionDecoder(Recurrent):

    def __init__(self, units, output_dim,
                 activation='tanh',
                 return_probabilities=False,
                 name='AttentionDecoder',
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        """
        Implements an AttentionDecoder that takes in a sequence encoded by an
        encoder and outputs the decoded states
        :param units: dimension of the hidden state and the attention matrices
        :param output_dim: the number of labels in the output space

        references:
            Bahdanau, Dzmitry, Kyunghyun Cho, and Yoshua Bengio.
            "Neural machine translation by jointly learning to align and translate."
            arXiv preprint arXiv:1409.0473 (2014).
        """
        self.units = units
        self.output_dim = output_dim
        self.return_probabilities = return_probabilities
        self.activation = activations.get(activation)
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionDecoder, self).__init__(**kwargs)
        self.name = name
        self.return_sequences = True  # must return sequences

    def build(self, input_shape):
        """
          See Appendix 2 of Bahdanau 2014, arXiv:1409.0473
          for model details that correspond to the matrices here.
        """

        self.batch_size, self.timesteps, self.input_dim = input_shape

        if self.stateful:
            super(AttentionDecoder, self).reset_states()

        self.states = [None, None]  # y, s

        """
            Matrices for creating the context vector
        """

        self.V_a = self.add_weight(shape=(self.units,),
                                   name='V_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.W_a = self.add_weight(shape=(self.units, self.units),
                                   name='W_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.U_a = self.add_weight(shape=(self.input_dim, self.units),
                                   name='U_a',
                                   initializer=self.kernel_initializer,
                                   regularizer=self.kernel_regularizer,
                                   constraint=self.kernel_constraint)
        self.b_a = self.add_weight(shape=(self.units,),
                                   name='b_a',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the r (reset) gate
        """
        self.C_r = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_r = self.add_weight(shape=(self.units, self.units),
                                   name='U_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_r = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_r',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_r = self.add_weight(shape=(self.units, ),
                                   name='b_r',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        """
            Matrices for the z (update) gate
        """
        self.C_z = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_z = self.add_weight(shape=(self.units, self.units),
                                   name='U_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_z = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_z',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_z = self.add_weight(shape=(self.units, ),
                                   name='b_z',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for the proposal
        """
        self.C_p = self.add_weight(shape=(self.input_dim, self.units),
                                   name='C_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_p = self.add_weight(shape=(self.units, self.units),
                                   name='U_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_p = self.add_weight(shape=(self.output_dim, self.units),
                                   name='W_p',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_p = self.add_weight(shape=(self.units, ),
                                   name='b_p',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)
        """
            Matrices for making the final prediction vector
        """
        self.C_o = self.add_weight(shape=(self.input_dim, self.output_dim),
                                   name='C_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.U_o = self.add_weight(shape=(self.units, self.output_dim),
                                   name='U_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.W_o = self.add_weight(shape=(self.output_dim, self.output_dim),
                                   name='W_o',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)
        self.b_o = self.add_weight(shape=(self.output_dim, ),
                                   name='b_o',
                                   initializer=self.bias_initializer,
                                   regularizer=self.bias_regularizer,
                                   constraint=self.bias_constraint)

        # For creating the initial state:
        self.W_s = self.add_weight(shape=(self.input_dim, self.units),
                                   name='W_s',
                                   initializer=self.recurrent_initializer,
                                   regularizer=self.recurrent_regularizer,
                                   constraint=self.recurrent_constraint)

        self.input_spec = [
            InputSpec(shape=(self.batch_size, self.timesteps, self.input_dim))]
        self.built = True

    def call(self, x):
        # store the whole sequence so we can "attend" to it at each timestep
        self.x_seq = x

        # apply the a dense layer over the time dimension of the sequence
        # do it here because it doesn't depend on any previous steps
        # thefore we can save computation time:
        self._uxpb = time_distributed_dense(self.x_seq, self.U_a, b=self.b_a,
                                             input_dim=self.input_dim,
                                             timesteps=self.timesteps,
                                             output_dim=self.units)

        return super(AttentionDecoder, self).call(x)

    def get_initial_state(self, inputs):
        # apply the matrix on the first time step to get the initial s0.
        s0 = activations.tanh(K.dot(inputs[:, 0], self.W_s))

        # from keras.layers.recurrent to initialize a vector of (batchsize,
        # output_dim)
        y0 = K.zeros_like(inputs)  # (samples, timesteps, input_dims)
        y0 = K.sum(y0, axis=(1, 2))  # (samples, )
        y0 = K.expand_dims(y0)  # (samples, 1)
        y0 = K.tile(y0, [1, self.output_dim])

        return [y0, s0]

    def step(self, x, states):

        ytm, stm = states

        # repeat the hidden state to the length of the sequence
        _stm = K.repeat(stm, self.timesteps)

        # now multiplty the weight matrix with the repeated hidden state
        _Wxstm = K.dot(_stm, self.W_a)

        # calculate the attention probabilities
        # this relates how much other timesteps contributed to this one.
        et = K.dot(activations.tanh(_Wxstm + self._uxpb),
                   K.expand_dims(self.V_a))
        at = K.exp(et)
        at_sum = K.sum(at, axis=1)
        at_sum_repeated = K.repeat(at_sum, self.timesteps)
        at /= at_sum_repeated  # vector of size (batchsize, timesteps, 1)

        # calculate the context vector
        context = K.squeeze(K.batch_dot(at, self.x_seq, axes=1), axis=1)
        # ~~~> calculate new hidden state
        # first calculate the "r" gate:

        rt = activations.sigmoid(
            K.dot(ytm, self.W_r)
            + K.dot(stm, self.U_r)
            + K.dot(context, self.C_r)
            + self.b_r)

        # now calculate the "z" gate
        zt = activations.sigmoid(
            K.dot(ytm, self.W_z)
            + K.dot(stm, self.U_z)
            + K.dot(context, self.C_z)
            + self.b_z)

        # calculate the proposal hidden state:
        s_tp = activations.tanh(
            K.dot(ytm, self.W_p)
            + K.dot((rt * stm), self.U_p)
            + K.dot(context, self.C_p)
            + self.b_p)

        # new hidden state:
        st = (1-zt)*stm + zt * s_tp

        yt = activations.softmax(
            K.dot(ytm, self.W_o)
            + K.dot(stm, self.U_o)
            + K.dot(context, self.C_o)
            + self.b_o)

        if self.return_probabilities:
            return at, [yt, st]
        else:
            return yt, [yt, st]

    def compute_output_shape(self, input_shape):
        """
            For Keras internal compatability checking
        """
        if self.return_probabilities:
            return (None, self.timesteps, self.timesteps)
        else:
            return (None, self.timesteps, self.output_dim)

    def get_config(self):
        """
            For rebuilding models on load time.
        """
        config = {
            'output_dim': self.output_dim,
            'units': self.units,
            'return_probabilities': self.return_probabilities
        }
        base_config = super(AttentionDecoder, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [22]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, Flatten, Embedding, Multiply, Concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import keras


def vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    
    print("Creating image model...")
    img_model = Sequential()
    img_model.add(Dense(1024, input_dim=4096, activation='relu'))

    image_input = Input(shape=(4096, ))
    encoded_image = img_model(image_input)

    print("Creating text model...")
    txt_model = Sequential()
    txt_model.add(Embedding(num_words, embedding_dim, 
        weights=[embedding_matrix], input_length=seq_length, trainable=False))
    txt_model.add(LSTM(units=256, return_sequences=False, input_shape=(seq_length, embedding_dim)))
    txt_model.add(Dropout(dropout_rate))
#     txt_model.add(AttentionDecoder(512, embedding_dim))
#     txt_model.add(LSTM(units=512, return_sequences=False))
#     txt_model.add(Dropout(dropout_rate))
#     txt_model.add(Dense(1024, activation='tanh'))
    
    question_input = Input(shape=(EMBEDDING_DIM, ), dtype='int32')
    embedded_question = txt_model(question_input)
    
    print(txt_model.summary())
    
    print("Merging final model...")
    merged = keras.layers.concatenate([encoded_image, embedded_question])
    d1  = Dense(1024, activation='relu')(merged)
    dp1 = Dropout(dropout_rate)(d1)
#     d2  = Dense(1000, activation='tanh')(dp1)
#     dp2 = Dropout(dropout_rate)(d2)
    output  = Dense(num_classes, activation='softmax')(dp1)
    
    vqa_model = Model(inputs=[image_input, question_input], outputs=output)
    
    
#     fc_model = Sequential()
#     # fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))
#     fc_model.add(Concatenate([img_model, txt_model]))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(1000, activation='tanh'))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(num_classes, activation='softmax'))
    
    vqa_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return vqa_model

In [25]:
dropout_rate=0.5
num_classes=26
model_weights_filename = "weights.bkp"
ckpt_model_weights_filename = "checkP.cp"

model = vqa_model(embedding_matrix, num_words, EMBEDDING_DIM, max_length_of_text, dropout_rate, num_classes)
if os.path.exists(model_weights_filename):
    print ("Loading Weights...")
    model.load_weights(model_weights_filename)
    
# print(model.summary())

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=ckpt_model_weights_filename,verbose=1)

batch_size = 64
model.fit_generator(
    generator(image_dict_V2, imageNamesX_train, questionsX_train, Y_train, batch_size),
    validation_data=generator(image_dict_V2, imageNamesX_test, questionsX_test, Y_test, batch_size),
    validation_steps=len(Y_test)/(batch_size-1), steps_per_epoch=len(Y_train)/(batch_size-1), nb_epoch=2, callbacks=[checkpointer])

Creating image model...
Creating text model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 200)          16200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 256)               467968    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
Total params: 484,168
Trainable params: 467,968
Non-trainable params: 16,200
_________________________________________________________________
None
Merging final model...


C:\Users\Anirudh\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\Anirudh\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., validation_steps=214.317460..., steps_per_epoch=1928.85714..., callbacks=[<keras.ca..., epochs=2)`


Epoch 1/2


ValueError: Error when checking input: expected input_5 to have 2 dimensions, but got array with shape (64, 100, 100, 3)

In [26]:
  #BEST
def vqa_model0(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model0
    model = vqa_model0.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [27]:
def vqa_model1(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model1
    model = vqa_model1.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [28]:
#ANet Experiment
def vqa_model3(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model3
    model = vqa_model3.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

In [29]:
dropout_rate=0.5
num_classes=26
model_weights_filename = "weightsV2.bkp"
ckpt_model_weights_filename = "checkpointsV2/checkP.cp"

model = None
if os.path.exists(ckpt_model_weights_filename):
    print("Loading model: {}".format(ckpt_model_weights_filename))
    from keras.models import load_model
    model = load_model(ckpt_model_weights_filename)
else:
    model = vqa_model0(embedding_matrix, num_words, EMBEDDING_DIM, max_length_of_text, dropout_rate, num_classes)
    if os.path.exists(model_weights_filename):
        print ("Loading Weights...")
        model.load_weights(model_weights_filename)
    
# print(model.summary())

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=ckpt_model_weights_filename, verbose=1)

batch_size = 10
model.fit_generator(
    generator(image_dict_V2, imageNamesX_train, questionsX_train, Y_train, batch_size),
    validation_data=generator(image_dict_V2, imageNamesX_test, questionsX_test, Y_test, batch_size),
    validation_steps=len(Y_test)/(batch_size-1), steps_per_epoch=len(Y_train)/(batch_size-1), nb_epoch=1, callbacks=[checkpointer])

ModuleNotFoundError: No module named 'vqa_model0'

In [0]:
def lstm(sequence_length, text_in):
  #text_in = Input(shape = (sequence_length,), name = 'text_in')
  text_in = text_in
  emb = Embedding(output_dim=256, input_dim=1024, input_length=64)(text_in)
  emb = LSTM(256)(emb)
  emb = Dropout(0.5)(emb)
  return emb

In [0]:
lstm = lstm(sequence_length, text_in)

In [0]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, Flatten, Embedding, Multiply, Concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
import keras


def vqa_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    
    print("Creating image model...")
    img_model = Sequential()
    img_model.add(Dense(256, input_dim=4096, activation='tanh'))

    image_input = Input(shape=(4096, ))
    encoded_image = img_model(image_input)

    print("Creating text model...")
    txt_model = Sequential()
    txt_model.add(Embedding(num_words, embedding_dim, 
        weights=[embedding_matrix], input_length=seq_length, trainable=False))
    txt_model.add(LSTM(units=256, return_sequences=False, input_shape=(seq_length, embedding_dim)))
    txt_model.add(Dropout(dropout_rate))
#     txt_model.add(AttentionDecoder(512, embedding_dim))
#     txt_model.add(LSTM(units=512, return_sequences=False))
#     txt_model.add(Dropout(dropout_rate))
#     txt_model.add(Dense(1024, activation='tanh'))
    
    question_input = Input(shape=(EMBEDDING_DIM, ), dtype='int32')
    embedded_question = txt_model(question_input)
    
    print(txt_model.summary())
    
    print("Merging final model...")
    merged = keras.layers.concatenate([encoded_image, embedded_question])
    d1  = Dense(1000, activation='softmax')(merged)
    dp1 = Dropout(dropout_rate)(d1)
#     d2  = Dense(1000, activation='tanh')(dp1)
#     dp2 = Dropout(dropout_rate)(d2)
    output  = Dense(num_classes, activation='softmax')(dp1)
    
    vqa_model = Model(inputs=[image_input, question_input], outputs=output)
    
    
#     fc_model = Sequential()
#     # fc_model.add(Merge([vgg_model, lstm_model], mode='mul'))
#     fc_model.add(Concatenate([img_model, txt_model]))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(1000, activation='tanh'))
#     fc_model.add(Dropout(dropout_rate))
#     fc_model.add(Dense(num_classes, activation='softmax'))
    
    vqa_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return vqa_model

In [0]:
dropout_rate=0.5
num_classes=26
model_weights_filename = "weights.bkp"
#ckpt_model_weights_filename = "checkpoints/checkP.cp"

model = vqa_model(embedding_matrix, num_words, EMBEDDING_DIM, max_length_of_text, dropout_rate, num_classes)
if os.path.exists(model_weights_filename):
    print ("Loading Weights...")
    model.load_weights(model_weights_filename)
    
# print(model.summary())

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath=ckpt_model_weights_filename,verbose=1)

batch_size = 128
model.fit_generator(
    generator(image_dict, imageNamesX_train, questionsX_train, Y_train, batch_size),
    validation_data=generator(image_dict, imageNamesX_test, questionsX_test, Y_test, batch_size),
    validation_steps=len(Y_test)/(batch_size-1), steps_per_epoch=len(Y_train)/(batch_size-1), nb_epoch=3, callbacks=[checkpointer])

In [0]:
## EXPERIMENT
from sklearn.preprocessing import MultiLabelBinarizer
labels = ["hello", "world", "hello", "cat"]
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

In [0]:
## ONE HOT
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

In [0]:
#BEST
def vqa_model0(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes):
    import vqa_model0
    model = vqa_model0.get_model(embedding_matrix, num_words, embedding_dim, seq_length, dropout_rate, num_classes)
    return model

### EXPERIMENTS ###

In [0]:
dropout_rate=0.5
num_classes=26
model_weights_filename = "weightsV2.bkp"
ckpt_model_weights_filename = "checkpointsV2/checkP.cp"

model = None
if os.path.exists(ckpt_model_weights_filename):
    print("Loading model: {}".format(ckpt_model_weights_filename))
    from keras.models import load_model
    model = load_model(ckpt_model_weights_filename)
else:
    model = vqa_model0(embedding_matrix, num_words, EMBEDDING_DIM, max_length_of_text, dropout_rate, num_classes)
    if os.path.exists(model_weights_filename):
        print ("Loading Weights...")
        model.load_weights(model_weights_filename)